In [ ]:
import pyarrow.parquet as pq
import pyarrow as pa
import glob
import numpy as np

In [ ]:
pa.__version__


In [ ]:
path_partition_breast_junctions  = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/commit_v3_TEST_merged3_372a147_medium_run_conf2_annotFrame_cap0_runs/selected_genes_rerun1/select_breast_junct_Expr'

In [ ]:
# Note the index of both segment and junction file should be the same. 
# To remove ALL, can junst remove one of them, regardless of junctions 
path_partition_GTEX_junctions  = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/commit_v3_TEST_merged3_372a147_medium_run_conf2_annotFrame_cap0_runs/selected_genes_rerun1/select_gtex_realign_junct_Expr'

In [ ]:
All_partition_GTEX_junctions = glob.glob(path_partition_GTEX_junctions + '/*')

In [ ]:
All_partition_breast_junctions = glob.glob(path_partition_breast_junctions + '/*')

In [ ]:
len(All_partition_GTEX_junctions)

In [ ]:
len(All_partition_breast_junctions)

In [ ]:
gtex_total_kmers = 0
gtex_total_kmers_rf_annot = 0 
set_gtex_kmers = set()
brca_concat = []

for idx in np.arange(len(All_partition_GTEX_junctions)):
    table_gtex = pq.read_table(All_partition_GTEX_junctions[idx], 
                               columns = ['kmer', 'readFrameAnnotated'])
    table_brca = pq.read_table(All_partition_breast_junctions[idx])
    pd_table_gtex = table_gtex.to_pandas()
    pd_table_brca = table_brca.to_pandas()

    gtex_total_kmers += pd_table_gtex.shape[0]
    pd_table_gtex = pd_table_gtex.loc[pd_table_gtex['readFrameAnnotated'] == 1.0]
    pd_table_brca = pd_table_brca.loc[pd_table_brca['readFrameAnnotated'] == 1.0]
    gtex_total_kmers_rf_annot += pd_table_gtex.shape[0]
    
    
    set_gtex = set(pd_table_gtex['kmer'])
    set_brca = set(pd_table_brca['kmer'])
    #reduce brca gene wise
    set_brca = set_brca.difference(set_gtex)
    brca_concat.append(pd_table_brca.set_index('kmer').loc[set_brca])
    set_gtex_kmers.update(set_gtex)
    print(idx)
